### **ID3 Algorithm**

**Import necessary modules:**

In [5]:
import numpy as np
import math
import csv

**Creation of Algorithmic Functions:**

In [4]:
def read_data(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)
        data = [row for row in reader]
    return headers, np.array(data)

In [6]:
class Node:
    def __init__(self, attribute="", answer=""):
        self.attribute = attribute
        self.answer = answer
        self.children = []

def entropy(S):
    unique_vals, counts = np.unique(S, return_counts=True)
    probs = counts / len(S)
    return -np.sum(probs * np.log2(probs))

def gain_ratio(data, col):
    total_entropy = entropy(data[:, -1])
    values, subsets = {}, np.unique(data[:, col])
    for value in subsets:
        subset = data[data[:, col] == value]
        values[value] = subset
    weighted_entropy = sum((len(v) / len(data)) * entropy(v[:, -1]) for v in values.values())
    intrinsic_value = -sum((len(v) / len(data)) * math.log2(len(v) / len(data)) for v in values.values())
    return (total_entropy - weighted_entropy) / intrinsic_value if intrinsic_value != 0 else 0


**Create Node**

In [7]:
def create_node(data, headers):
    if len(np.unique(data[:, -1])) == 1:  # If all instances have the same label
        return Node(answer=np.unique(data[:, -1])[0])

    gains = [gain_ratio(data, i) for i in range(data.shape[1] - 1)]
    best_col = np.argmax(gains)
    node = Node(attribute=headers[best_col])

    headers = np.delete(headers, best_col)
    for value in np.unique(data[:, best_col]):
        subset = data[data[:, best_col] == value]
        child_node = create_node(subset, headers)
        node.children.append((value, child_node))

    return node


**Print Tree**

In [8]:
def print_tree(node, depth=0):
    if node.answer:
        print("  " * depth + node.answer)
    else:
        print("  " * depth + node.attribute)
        for value, child in node.children:
            print("  " * (depth + 1) + str(value))
            print_tree(child, depth + 2)


**Main Execution**

In [10]:
headers, data = read_data("/content/drive/MyDrive/Tennisdata.csv")
tree = create_node(data, headers)
print_tree(tree)

Outlook
  Overcast
    Yes
  Rain
    Play Tennis
      Strong
        No
      Weak
        Yes
  Sunny
    Wind
      High
        No
      Normal
        Yes
